In [ ]:


import pandas as pd


fakedata = pd.read_csv('/content/Lots_Permissions_CH5_fakedata (1).csv')
lot_names = set(fakedata['Lot Name'])



def isSliceCyclc(slice):
  print(slice.loc[0, 'Start Time - Daily'], slice.loc[0, 'End Time - Daily'])
  if slice.loc[0, 'Start Time - Daily'] == slice.loc[0, 'End Time - Daily']:
    flag = True
  else:
    flag = False

  start = slice.loc[0, 'Start Time - Daily']
  print(start)
  for idx, row in slice.iloc[1:,:].iterrows():
    print(row['Start Time - Daily'],row['End Time - Daily'])
    if row['End Time - Daily'] == start:
      flag = True
      break

    if row['Start Time - Daily'] != slice.loc[idx-1, 'End Time - Daily']:
      flag = False
      break

  return flag

def check_cyclic_lots(data, lot_names):
  cyclic_lots_weekdays = []
  non_cyclic_lots_weekdays = []
  cyclic_lots_weekends = []
  non_cyclic_lots_weekends = []


  for lot_name in lot_names:
    print(lot_name)
    slice = data[(data['Lot Name']==lot_name) & (data['Enforcement Days']=='Weekdays')].reset_index()
    if len(slice)!=0:
      if isSliceCyclc(slice):
        cyclic_lots_weekdays.append(lot_name)
      else:
        non_cyclic_lots_weekdays.append(lot_name)
    slice = data[(data['Lot Name']==lot_name) & (data['Enforcement Days']=='Weekends')].reset_index()
    if len(slice)!=0:
      if isSliceCyclc(slice):
        cyclic_lots_weekends.append(lot_name)
      else:
        non_cyclic_lots_weekends.append(lot_name)
    slice = data[(data['Lot Name']==lot_name) & (data['Enforcement Days']=='Always')].reset_index()
    if len(slice)!=0:
      if isSliceCyclc(slice):
        cyclic_lots_weekends.append(lot_name)
      else:
        non_cyclic_lots_weekends.append(lot_name)
  return cyclic_lots_weekdays, non_cyclic_lots_weekdays, cyclic_lots_weekends, non_cyclic_lots_weekends

cyclic_lots_weekdays, non_cyclic_lots_weekdays, cyclic_lots_weekends, non_cyclic_lots_weekends = check_cyclic_lots(fakedata, lot_names)
print(non_cyclic_lots_weekdays, non_cyclic_lots_weekends)



import pandas as pd

def parse_parking_data(df):
    """
    Given a DataFrame 'df' with columns:
        - "Lot Type"
        - "Physical Location (Yes/No)"
        - "Parking Lot / Zone Name"
        - "Posted Restrictions"
        - "Enforcement Days"
        - "Start Time - Daily"
        - "End Time - Daily"
        - ... plus many permit columns (e.g., "17FAE", "A", "AA", etc.)
    Return a nested dictionary in the format:

    {
        lot_name: {
            "Type": <string>,
            "Physical Location": <bool>,
            "Permissions": {
                day_type (e.g. "Weekdays"): {
                    (start_time, end_time): {
                        permit_name: bool,
                        ...
                    }
                }
            }
        },
        ...
    }
    """

    # Identify which columns are permits by excluding known metadata columns
    known_columns = {
        "Lot Type ",
        "Physical Location (Yes/No)",
        "Lot Name",
        "Posted Restrictions",
        "Enforcement Days",
        "Start Time - Daily",
        "End Time - Daily",
        "Count Valid Permissions in Lot by Date/Time",
    }

    # All other columns are presumably permit columns
    permit_columns = [col for col in df.columns if col not in known_columns]

    # Our final nested dictionary
    lots_dict = {}

    for _, row in df.iterrows():
        lot_name = str(row["Lot Name"])
        lot_type = str(row["Lot Type "])
        physical_location_val = str(row["Physical Location (Yes/No)"]).strip().upper()
        # Convert "YES"/"NO" to boolean
        physical_location_bool = (physical_location_val == "YES")

        enforcement_day = str(row["Enforcement Days"]).strip()
        start_time_raw = str(row["Start Time - Daily"]).strip()
        end_time_raw = str(row["End Time - Daily"]).strip()

        # If your dataset uses "0:00:00" to mean midnight, you might
        # want to convert to "00:00" or "24:00" for clarity. For example:
        # start_time = "00:00" if start_time_raw == "0:00:00" else start_time_raw
        # But here, we just keep them as-is or do minimal cleanup:
        start_time = start_time_raw
        end_time = end_time_raw

        # Initialize lot entry if not present
        if lot_name not in lots_dict:
            lots_dict[lot_name] = {
                "Type": lot_type,
                "Physical Location": physical_location_bool,
                "Permissions": {}
            }

        # Prepare to store the permit booleans
        permit_dict = {}
        for pcol in permit_columns:
            val = row[pcol]
            # Convert 1 -> True, 0 -> False (or strings "1"/"0" similarly)
            permit_dict[pcol] = bool(val)

        # Insert into the nested structure
        if enforcement_day not in lots_dict[lot_name]["Permissions"]:
            lots_dict[lot_name]["Permissions"][enforcement_day] = {}

        # Use (start_time, end_time) as a tuple key
        time_tuple = (start_time, end_time)
        lots_dict[lot_name]["Permissions"][enforcement_day][time_tuple] = permit_dict

    return lots_dict


# ------------------------------

import pandas as pd
from datetime import datetime, timedelta, time  # Import the time class

# Convert 'End Time - Daily' to datetime objects
fakedata['End Time - Daily'] = pd.to_datetime(fakedata['End Time - Daily'], format='%H:%M:%S').dt.time

# Reduce all 'End Time - Daily' times by 1 second, handling midnight case
def subtract_one_second(t):
    if t == datetime.min.time():  # Check if it's midnight
        return time(23, 59, 59)  # Use time class to create time object
    else:
        return (datetime.combine(datetime.min, t) - timedelta(seconds=1)).time()

fakedata['End Time - Daily'] = fakedata['End Time - Daily'].apply(subtract_one_second)

# Convert 'End Time - Daily' back to string format
fakedata['End Time - Daily'] = fakedata['End Time - Daily'].astype(str)

parsed_data = parse_parking_data(fakedata)

import pandas as pd
from collections import Counter

def make_prefix_mapping_from_df(df, prefix_col="Prefix", perm_col="Permissions"):
    """
    Build a dictionary from a 2-column DataFrame with columns:
      prefix_col, perm_col
    Rules:
      - If prefix appears exactly once in the DF, map prefix -> permission
      - If prefix appears multiple times, map prefix-permission -> permission
    """
    # 1) Collect (prefix, permission) pairs
    pairs = []
    for _, row in df.iterrows():
        prefix = str(row[prefix_col]).strip()
        perm   = str(row[perm_col]).strip()
        # Only add if there's something
        if prefix or perm:
            pairs.append((prefix, perm))

    # 2) Count how many times each prefix occurs
    prefix_counts = Counter(pair[0] for pair in pairs)

    # 3) Build the dictionary
    mapping = {}
    seen = set()
    for prefix, perm in pairs:
        if (prefix, perm) in seen:
            # skip duplicates if the exact (prefix, perm) repeats
            continue
        seen.add((prefix, perm))

        if prefix_counts[prefix] == 1:
            # Unique prefix => map prefix -> permission
            mapping[prefix] = perm
        else:
            # Duplicate prefix => map "prefix-permission" -> permission
            mapping[f"{prefix}-{perm}"] = perm

    return mapping

# ------------------------------
# Example usage:
if __name__ == "__main__":
    # Suppose you have a DataFrame named df_prefix with two columns:
    # e.g. df_prefix.columns = ["Prefix", "Permission"]
    #
    # For demonstration, let's build a small example DataFrame:

    data = pd.read_csv('/content/Permits & Permissions.csv')
    df_prefix = data[['Prefix', 'Permissions']]

    # Build the dictionary
    map_prefix_to_permission = make_prefix_mapping_from_df(df_prefix)

    # Print in your requested format
    print("map_prefix_to_permission = {")
    for k, v in map_prefix_to_permission.items():
        print(f"    '{k}': '{v}',")
    print("}")


import pandas as pd
from collections import Counter


# Check which values in map_prefix_to_permission are not columns in fakedata
missing_cols = []
for prefix in map_prefix_to_permission.values():
    if prefix not in fakedata.columns:
        missing_cols.append(prefix)

print("Columns in map_prefix_to_permission not found in fakedata:", missing_cols)

# Assuming 'data_hella_new' DataFrame is already loaded

def find_non_unique_prefixes(df, column_name='Prefix'):
    """
    Finds and returns a list of non-unique values in the specified column of a DataFrame.
    """
    value_counts = df[column_name].value_counts()
    non_unique_values = value_counts[value_counts > 1].index.tolist()
    return non_unique_values

# Assuming data_hella_new is your DataFrame
non_unique_prefixes = find_non_unique_prefixes(data, column_name='Prefix')

def is_time_in_tuple(time_str, time_tuple):
    """
    Checks if a time string is within a given time tuple.

    Args:
        time_str: The time string to check (e.g., "08:30:00").
        time_tuple: A tuple containing two time strings representing the start and end times (e.g., ("07:00:00", "16:00:00")).

    Returns:
        True if the time string falls within the time tuple (inclusive), False otherwise.
    """
    try:
        time_obj = datetime.strptime(time_str, "%H:%M:%S").time()
        start_time_obj = datetime.strptime(time_tuple[0], "%H:%M:%S").time()
        end_time_obj = datetime.strptime(time_tuple[1], "%H:%M:%S").time()

        return start_time_obj <= time_obj <= end_time_obj
    except ValueError:
        return False # Handle invalid time string format


def extract_prefix(permit_no: str) -> str: #Assumption: 5 numbers after prefix
  return permit_no[:-5]


day_mapper = {
    'monday': 'Weekdays',
    'tuesday': 'Weekdays',
    'wednesday': 'Weekdays',
    'thursday': 'Weekdays',
    'friday': 'Weekdays',
    'saturday': 'Weekends',
    'sunday': 'Weekends'
}

def number_letter_extract(number_letter):
  s = ""
  for idx in range(len(number_letter)-1, -1, -1):
    try:
      s = number_letter[:idx]
      return int(s)

    except:
      continue



def extract_number_from_lot(lot: str)->str:
  if 'Lot' in lot:
    the_rest = lot[4:]

    return number_letter_extract(the_rest)


In [ ]:
!pip install faiss-cpu

In [ ]:
from sentence_transformers import SentenceTransformer
import numpy as np
import faiss

def find_lot_name(input_name: str, lot_names: set) -> dict:
    """
    Takes a string `input_name` and a set of known lot names `lot_names`.
    If `input_name` is exactly in `lot_names`, returns [input_name].
    Otherwise, it returns a list of the 5 closest lot names.
    """
    # 1) Quick exact check
    if input_name in lot_names:
        return {1: input_name}

    # 2) Build a list from the set so we can index it
    lot_names_list = list(lot_names)

    # 3) Load a sentence embedding model (or any other embedding method you prefer)
    model = SentenceTransformer("all-MiniLM-L6-v2")

    # 4) Compute embeddings for all known lot names
    lot_embeddings = model.encode(lot_names_list, convert_to_numpy=True)

    # 5) Build a FAISS index
    embed_dim = lot_embeddings.shape[1]  # Dimension of the embeddings
    index = faiss.IndexFlatL2(embed_dim)
    index.add(lot_embeddings)

    # 6) Compute embedding for input_name
    input_embedding = model.encode([input_name], convert_to_numpy=True)

    # 7) Search for the top 5 matches
    k = 5
    distances, indices = index.search(input_embedding, k)

    # 8) Return the 5 closest names
    closest_matches = {i+1: lot_names_list[idx] for i, idx in enumerate(indices[0])}
    return closest_matches

In [ ]:
def check_parking_eligibility():
  global non_unique_prefixes
  global map_prefix_to_permission
  global parsed_data
  global lot_names
  global day_mapper

  permit = str(input("Enter permit number: "))
  lot = str(input("Enter lot name: "))
  day = str(input("Enter day: "))
  time = str(input("Enter time: "))


  if lot[0] in "1234567890":
    lot = f'Lot {lot}'

  lot_results = find_lot_name(lot, lot_names)

  if len(lot_results) == 1:
    lot = lot_results[1]
  else:
    print(f"Lot not found. Closest Matches:\n{lot_results}")
    lot_key = input('Enter Key: ')
    lot = lot_results[int(lot_key)]

  if 'Lot' in lot:
    num = extract_number_from_lot(lot)
    lot_perm = 'Lot ' + str(num)
  else: lot_perm = lot

  # print (lot)
  if lot not in lot_names:
    if f'Lot {lot}' in lot_names:
      lot = f'Lot {lot}'
    else:
      print("Lot not found in data.")
      return

  prefix = extract_prefix(permit)
  if prefix in non_unique_prefixes:
    permit_perm = str(input(f"Enter Lot you have this {prefix} permit for: "))
    new_prefix = f'{prefix}-{permit_perm}'
  else:
    new_prefix = prefix
  if new_prefix in map_prefix_to_permission:
    perm = map_prefix_to_permission[new_prefix]
  else:
    # if prefix not in map_prefix_to_permission:
    #   if f'{prefix}-{lot_perm}' in map_prefix_to_permission:
    #     perm = map_prefix_to_permission[f'{prefix}-{lot}']
    #   else:
    #     print(f'{prefix}-{lot_perm}')
    #     print('Invalid Permit Input')
    #     return
    closest_match = find_lot_name(new_prefix, set(map_prefix_to_permission.keys()))
    if len(closest_match) == 1:
      perm = map_prefix_to_permission[closest_match[1]]
    else:
      print(f"Did you mean:\n{closest_match}")
      perm_key = input('Enter Key: ')
      perm = map_prefix_to_permission[closest_match[int(perm_key)]]


  if day.lower() in day_mapper:
    day = day_mapper[day.lower()]
  else:
    print('Invalid Day Input')
    return


  available_days = list(parsed_data[lot]['Permissions'].keys())
  if day not in available_days:
    day = 'Always'

  available_times = list(parsed_data[lot]['Permissions'][day].keys())

  for time_tuple in available_times:
    if is_time_in_tuple(time, time_tuple):
      print(f'Parking Permission for {lot} with permit {perm} is {parsed_data[lot]["Permissions"][day][time_tuple][perm]}')
      return
  print(f'Parking Permission for {lot} with permit {perm} is not abailable')
